In [84]:
import random

In [36]:
import sys
sys.path.append('../src')
sys.path.append('../src/data/')
sys.path.append('../src/models/')
sys.path.append('../src/features/')
sys.path.append('../src/visualization/')
# will reload any library
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from data.metrics import recall,r_precision
from visualization.plot_utils import write_latex_table, summary_pooling_table, pooling_plots, pandas_settings, plot_settings,  save_result
# general
import pandas as pd
import numpy as np
from tqdm import tqdm
from features.tracks_info import TrackInfo
from collections import OrderedDict
from itertools import product
from data_utils import product_size
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from features.tracks_info import TrackInfo
from features.popularity_feature import Popularity

tracks = TrackInfo(subset='5k-')
popularity = Popularity()

In [38]:
import json
from random import randint
import os
import json
import lyricwikia
import pickle
import sys
import numpy as np

def artists_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['artist_name'])
    return list(set(all_))

def albums_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['album_name'])
    return list(set(all_))

def tracks_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['track_uri'])
    return list(set(all_))

def track_names_in_playlist(playlist):
    all_ = []
    for track in playlist['tracks']:
        all_.append(track['track_name'])
    return list(set(all_))

def lyrics_in_playlist(playlist):
    all_ = {}
    info = {}
    ct = 0
    failed = []
    for track in playlist['tracks']:
        track_uri = track['track_uri']
        song_name = track['track_name']
        artist_name = track['artist_name']
        try:
            lyrics = lyricwikia.get_lyrics(artist_name, song_name)
            all_[track_uri] = lyrics
            info[track_uri] = (song_name,artist_name)
        except:
            failed.append(ct)
        ct += 1
    return all_,failed,info

f = open('../data/pooling/uri_to_track_info.pckl','rb')
uri_to_name_artist = pickle.load(f)
f.close()

f = open('../data/pooling/artist_to_songs.p','rb')
artist_to_songs = pickle.load(f)
f.close()

clean_artist_to_songs = {}
for artist in artist_to_songs.keys():
    clean_artist_to_songs[artist] = list(set(artist_to_songs[artist]))

name_artist_to_uri = {}
for uri in uri_to_name_artist.keys():
    tup = uri_to_name_artist[uri]
    name_artist_to_uri[tup] = uri

In [39]:
# from gensim.models import Word2Vec
# model = Word2Vec.load('../data/pooling/model_min5_new.bin')

# Open Pooling model results

In [119]:
sum_recall = 0
sum_rprec = 0
playlist_count = 50
for pid  in range(playlist_count):
    pl_turi,pl_auri = tracks.get_playlist(pid)
    train_uri = pl_turi[:int(len(pl_turi)*0.75)]
    test_uri = pl_turi[int(len(pl_turi)*0.75):]
    f = open('../data/pooling/pool_songs_pid_%s.p'%pid,'rb')
    songs_pool = pickle.load(f)
    preds = [i[0].split(':')[-1] for i in songs_pool][:10000]
#     random.shuffle(preds)
    ##############
#     random.shuffle(preds)
#     preds = preds[:len(pl_turi)*100]
    ##############
    
    preds_names = [uri_to_name_artist[i[0]] for i in songs_pool]
    f.close()
    sum_recall += recall(test_uri,preds)
    sum_rprec += r_precision(test_uri,preds)
#     print('Recall: %s'%recall(test_uri,preds))
#     print('R-Precision: %s'%r_precision(test_uri,preds))
print(sum_recall/float(playlist_count))
print(sum_rprec/float(playlist_count))

0.7023830278818068
0.02859874975148063


In [106]:
preds_names[:10]

[('Big Bird - Mono', 'Eddie Floyd'),
 ('I Want to Take You Higher', 'Sly & The Family Stone'),
 ('Gimme Shelter', 'Merry Clayton'),
 ('Rumble', 'Link Wray'),
 ('96 Tears', '? & The Mysterians'),
 ('Jungle Fever', 'The Chakachas'),
 ('a) STRA (Illusions Of My Childhood-Part One) b)You Keep Me Hanging On c) WBER (Illusions Of My Childhood-Part Two)',
  'Vanilla Fudge'),
 ('Reach Out Of The Darkness (Re-Recorded / Remastered)', 'Friend & Lover'),
 ('Bullwinkle Pt. II', 'The Centurians'),
 ('Pretty Thing', 'Bo Diddley')]

In [82]:
train_songs

[('Take My Side', 'Will Butler'),
 ('Everyday it Starts', 'Parquet Courts'),
 ('Content Nausea', 'Parquet Courts'),
 ('Urban Ease', 'Parquet Courts'),
 ('Slide Machine', 'Parquet Courts'),
 ('Kevlar Walls', 'Parquet Courts'),
 ('Pretty Machines', 'Parquet Courts'),
 ('Psycho Structures', 'Parquet Courts'),
 ('The Map', 'Parquet Courts'),
 ("These Boots Are Made for Walkin'", 'Parquet Courts'),
 ('Insufferable', 'Parquet Courts'),
 ('No Concept', 'Parquet Courts'),
 ('Uncast Shadow of a Southern Myth', 'Parquet Courts'),
 ('Bodies Made Of', 'Parquet Courts'),
 ('Black And White', 'Parquet Courts'),
 ('Dear Ramona', 'Parquet Courts'),
 ('What Color Is Blood', 'Parquet Courts'),
 ('Vienna II', 'Parquet Courts'),
 ('Always Back In Town', 'Parquet Courts'),
 ("She's Rolling", 'Parquet Courts'),
 ('Sunbathing Animal', 'Parquet Courts'),
 ('Up All Night', 'Parquet Courts'),
 ('Instant Disassembly', 'Parquet Courts'),
 ('Ducking And Dodging', 'Parquet Courts'),
 ('Raw Milk', 'Parquet Courts'),

In [81]:
train_songs = [uri_to_name_artist['spotify:track:'+uri] for uri in train_uri]
test_songs = [uri_to_name_artist['spotify:track:'+uri] for uri in test_uri]
test_songs[:min(10,len(test_songs))]

[('Separate Songs', 'Restorations'),
 ('Misprint', 'Restorations'),
 ('Tiny Prayers', 'Restorations'),
 ('All My Home', 'Restorations'),
 ('Most Likely A Spy', 'Restorations'),
 ('No Castle', 'Restorations'),
 ('The Future', 'Restorations'),
 ("It's Not", 'Restorations'),
 ('D', 'Restorations'),
 ("Let's Blow Up The Sun", 'Restorations')]